In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel("C:/Users/JngyEn/Downloads/2024/B题/表1test.xlsx")
df.columns = df.columns.str.replace(' ', '', regex=True)
print(df.columns)

df.head()

Index(['情况', '零配件1次品率', '零配件1购买单价', '零配件1检测成本', '零配件2次品率', '零配件2购买单价',
       '零配件2检测成本', '成品次品率', '成品装配成本', '成品检测成本', '成品市场售价', '调换费用', '拆解费用'],
      dtype='object')


,情况,零配件1次品率,零配件1购买单价,零配件1检测成本,零配件2次品率,零配件2购买单价,零配件2检测成本,成品次品率,成品装配成本,成品检测成本,成品市场售价,调换费用,拆解费用
0,1,0.1,4,2,0.1,18,3,0.1,6,3,56,6,5
1,2,0.2,4,2,0.2,18,3,0.2,6,3,56,6,5
2,3,0.1,4,2,0.1,18,3,0.1,6,3,56,30,5
3,4,0.2,4,1,0.2,18,1,0.2,6,2,56,30,5
4,5,0.1,4,8,0.2,18,1,0.1,6,2,56,10,5


In [103]:
# 定义一个函数，根据不同的情况选择对应的参数
def get_parameters(case_number):
    global a1, a2, a3, b1, b2, b3, c1, c2, c3, c4, d1, d2  # 声明要修改的全局变量
    # 从表格中读取对应行的数据，case_number 是情况编号（1到6）
    row = df.iloc[case_number - 1]
    
    # 零配件1的次品率、检测成本、购买单价
    a1 = row['零配件1次品率']   # 零配件1次品率
    a2 = row['零配件1检测成本']  # 零配件1检测成本
    a3 = row['零配件1购买单价']  # 零配件1购买单价

    # 零配件2的次品率、检测成本、购买单价
    b1 = row['零配件2次品率']   # 零配件2次品率
    b2 = row['零配件2检测成本']  # 零配件2检测成本
    b3 = row['零配件2购买单价']  # 零配件2购买单价

    # 成品的次品率、检测成本、装配成本、市场售价
    c1 = row['成品次品率']       # 成品次品率
    c2 = row['成品检测成本']     # 成品检测成本
    c3 = row['成品装配成本']     # 成品装配成本
    c4 = row['成品市场售价']     # 成品市场售价

    # 不合格成品的调换损失和拆解费用
    d1 = row['调换费用']         # 调换损失
    d2 = row['拆解费用']         # 拆解费用


In [104]:
import pulp

# 定义问题
problem = pulp.LpProblem("Production_Decision", pulp.LpMinimize)

# 定义变量 0 为否，1 为是
x1 = pulp.LpVariable('x1', cat='Binary')  # 零配件1是否检测
x2 = pulp.LpVariable('x2', cat='Binary')  # 零配件2是否检测
x3 = pulp.LpVariable('x3', cat='Binary')  # 成品是否检测
x4 = pulp.LpVariable('x4', cat='Binary')  # 不合格成品是否拆解

In [105]:
N = 10000  # 假设总成品量

In [106]:

# 参数设置，初始化为情况1的
# 零配件1的次品率、检测成本、购买单价
a1 = 0.1  # 零配件1的次品率
a2 = 2    # 零配件1的检测成本
a3 = 4    # 零配件1的购买单价

# 零配件2的次品率、检测成本、购买单价
b1 = 0.2  # 零配件2的次品率
b2 = 3    # 零配件2的检测成本
b3 = 18   # 零配件2的购买单价

# 成品的次品率、检测成本、装配成本、市场售价
c1 = 0.1  # 成品次品率
c2 = 3    # 成品检测成本
c3 = 6    # 成品装配成本
c4 = 56   # 成品市场售价

# 不合格成品的调换损失和拆解费用
d1 = 6    # 调换损失
d2 = 5    # 拆解费用

# 假设值
x1 = 0  # 零配件1是否检测
x2 = 0  # 零配件2是否检测
x3 = 0  # 成品是否检测
x4 = 0  # 不合格成品是否拆解
get_parameters(4)





In [107]:
# 组装后的次品的概率
# 良品率
c_good = (1 - a1 * (1 - x1)) * (1 - b1 * (1 - x2)) * (1 - c1 * (1 - x3))
c5 = 1 - c_good

# 最终次品个数
c_defective_num = c5 * N 

# 最终成品个数
c_sell_num = N - c_defective_num

print ("组装后的次品的概率:",c5)
print ("组装后的次品的个数:",c_defective_num)
print ("组装后的成品的个数:",c_sell_num)

组装后的次品的概率: 0.4879999999999999
组装后的次品的个数: 4879.999999999999
组装后的成品的个数: 5120.000000000001


# 部分函数定义

## 零配件1

In [108]:
# 采购数量
a_N = N / (1-a1 * (1 - x1))

# 采购成本
a_cost = a3 * a_N

# 检测成本
a_detect_cost =  x1 * a2 * N / (1 - a1)

## 零部件2

In [109]:
# 采购数量
b_N = N / (1-b1 * (1 - x2))

# 采购成本
b_cost = b3 * b_N

# 检测成本
b_detect_cost = b2 * N / (1 - b1) * x2
print(b_detect_cost)

0.0


## 成品

In [110]:
# 装配价格
c_cost = N * c3

In [111]:

# 检测价格
c_detect_cost = N * c2 * x3


In [112]:
# 调换损失
c_lose = (1 - x3)  * c_defective_num * d1 

In [113]:
# ================================== 仔细斟酌 ================================
# 拆解成本，零件1 检测之后，肯定没有次品，那么拆出来的零件1 就不需要检测了
c_dismantle_cost = x4 * ((d2+c3) - (a3 + b3) + ((1 - x1) * a2 + (1 - x2)*b2 + (1-x3)*c2))*c_defective_num
print("拆解计算：",((d2+c3) - (a3 + b3) + ((1 - x1) * a2 + (1 - x2)*b2 + (1-x3)*c2)))

拆解计算： -7.0


目前的想法是，既然是调换，那么我给他一个好的，他还我一个坏的
我同样要生产出成品之后再销售，相当于省掉了一笔原料购买的费用

## 总成本和总利润

In [114]:
Total_cost = a_cost + a_detect_cost + b_cost + b_detect_cost + c_cost + c_detect_cost +  c_lose + c_dismantle_cost

Total_profit = c_sell_num * c4 - Total_cost


In [115]:
print("x1:", x1, "x2:", x2, "x3:", x3, "x4:", x4)
variables = {
    'a1': a1, 'a2': a2, 'a3': a3,
    'b1': b1, 'b2': b2, 'b3': b3,
    'c1': c1, 'c2': c2, 'c3': c3, 'c4': c4,
    'd1': d1, 'd2': d2,
    'c_good': c_good, 'c5': c5,
    'c_defective_num': c_defective_num, 'c_sell_num': c_sell_num,
    'a_N': a_N, 'a_cost': a_cost, 'a_detect_cost': a_detect_cost,
    'b_N': b_N, 'b_cost': b_cost, 'b_detect_cost': b_detect_cost,
    'c_cost': c_cost, 'c_detect_cost': c_detect_cost,
    'c_lose': c_lose, 'c_dismantle_cost': c_dismantle_cost,
    'Total_cost': Total_cost, 
    'Total_profit': Total_profit
}
    
for key, value in variables.items():
    print(f"{key}: {value}")

x1: 0 x2: 0 x3: 0 x4: 0
a1: 0.2
a2: 1.0
a3: 4.0
b1: 0.2
b2: 1.0
b3: 18.0
c1: 0.2
c2: 2.0
c3: 6.0
c4: 56.0
d1: 30.0
d2: 5.0
c_good: 0.5120000000000001
c5: 0.4879999999999999
c_defective_num: 4879.999999999999
c_sell_num: 5120.000000000001
a_N: 12500.0
a_cost: 50000.0
a_detect_cost: 0.0
b_N: 12500.0
b_cost: 225000.0
b_detect_cost: 0.0
c_cost: 60000.0
c_detect_cost: 0.0
c_lose: 146399.99999999997
c_dismantle_cost: -0.0
Total_cost: 481400.0
Total_profit: -194679.99999999994


In [116]:
print(f'a_cost: {a_cost}')
print(f'a_detect_cost: {a_detect_cost}')
print(f'b_cost: {b_cost}')
print(f'b_detect_cost: {b_detect_cost}')
print(f'c_cost: {c_cost}')
print(f'c_detect_cost: {c_detect_cost}')
print(f'c_lose: {c_lose}')
print(f'c_dismantle_cost: {c_dismantle_cost}')
print(f'c_sell: {c_sell_num}')
print(f'c4: {c4}')
print(f'Total_cost: {Total_cost}')
print(f'Total_profit: {Total_profit}')

a_cost: 50000.0
a_detect_cost: 0.0
b_cost: 225000.0
b_detect_cost: 0.0
c_cost: 60000.0
c_detect_cost: 0.0
c_lose: 146399.99999999997
c_dismantle_cost: -0.0
c_sell: 5120.000000000001
c4: 56.0
Total_cost: 481400.0
Total_profit: -194679.99999999994
